*PLACES RECOMMENDATION ENGINE!*

# Load data



In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load data
places_df = pd.read_csv('places.csv')
places_df

# Preprocess data

In [ ]:
places_df['categories'] = places_df['categories'].apply(lambda x: x.split(';'))
categories = set([cat for cats in places_df['categories'] for cat in cats])
categories

In [7]:
for cat in categories:
    places_df[cat] = places_df['categories'].apply(lambda x: int(cat in x))

In [8]:
places_df = places_df.drop(['categories'], axis=1)

In [ ]:
# Compute item similarity
cosine_sim = cosine_similarity(places_df.drop(['place_id', 'place_name'], axis=1))

# Define function to recommend places based on user preferences


In [ ]:
def recommend_places(user_preferences, n_recommendations=5):
    # Create user profile vector
    user_profile = pd.Series(index=categories, data=0)
    for pref in user_preferences:
        user_profile[pref] = 1
    
    # Compute user similarity to items
    user_sim = cosine_similarity([user_profile], places_df.drop(['place_id', 'place_name'], axis=1))
    
    # Get top n similar items
    sim_scores = list(enumerate(user_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n_recommendations+1]
    
    # Get recommended places
    place_indices = [i[0] for i in sim_scores]
    recommendations = places_df[['place_name', 'categories']].iloc[place_indices].values.tolist()
    
    return recommendations